# ANALISIS DE SERIES DE TIEMPO EN EL MERVAL

Este analisis va a determinar si el indice MERVAL tiene un comportamiento de **random walk, reversion a la media o tendencia.**

Cabe destacar que los datos utilizados parten desde **octubre 1996 hasta septiembre 2018**.

### Importamos las librerias necesarias para el analisis

In [1]:
from __future__ import print_function

In [2]:
import statsmodels.tsa.stattools as ts

C:\Users\david\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
from datetime import datetime
import pandas as pd
import numpy as np

### Observamos si el directorio esta bien determinado (donde se encuentra nuestro archivo CSV)

In [4]:
cd C:\Users\david\Documents\Datasets

C:\Users\david\Documents\Datasets


### Creamos una variable llamada MERVAL y cargamos los datos del CSV

Creamos una segunda variable llamada MERV que va a contener solo los valores de cierre y dejamos afuera aquellos valores que faltan o son clasificados como NaN. Para eso utilizamos el **.dropna()** 

In [5]:
MERVAL = pd.read_csv("MERV.csv")

In [6]:
MERV = MERVAL['Adj Close'].dropna()

### Realizamos el Augmented Dickey-Fuller (ADF) Test

* **Valor 1:** test estadistico calculado.
* **Valor 2:** p-value.
* **Valor 4:** Numero de observaciones en la muestra.
* **Valor 5:** Diccionario que contiene los valores criticos del test en 1%, 5% y 10%.

In [7]:
ts.adfuller(MERV, 1)

(1.795529925159729,
 0.9983401100418371,
 1,
 5369,
 {'1%': -3.431568556383375,
  '10%': -2.567056631251383,
  '5%': -2.862078478299543},
 70529.76339805691)

Debido a que el valor calculado del test estaditico es **mayor** que cualquiera de los valores criticos en los niveles porcentuales de 1, 5 y 10, **no podemos rechazar la hipotesis nula de γ = 0**. Por consecuente, el **MERVAL no tiene un comportamiento de reversion a la media.**

### Importamos librerias faltantes para testear la Estacionariedad de la serie

En una serie estacionaria, la media y varianza del proceso no cambia según tiempo y espacio (se mantienen constantes) y cada una no sigue tendencia alguna. Una característica de las series de precios estacionarias es que sus precios se dispersan de su valor inicial a un ritmo más lento que la del GBM. Calculando esta diffuse rate, podemos intendificar la naturaleza de la serie de tiempo y detectar si hay reversión a la media.

In [8]:
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn

### Definimos la funcion para calcular el exponente de Hurst:

Mediante el HE vamos a poder obtener un valor escalar que va a permitir identificar si la serie tiene una conducta de reversión a la media, random walk o tendencia. Se utiliza la varianza de una serie de tiempo logarítmica para evaluar la tasa del comportamiento difusivo.

In [9]:
def hurst(ts):
    # Creamos el rango de valores laggeados
    lags = range(2, 100)
    # Calculamos la matriz de las varianzas de la diferencias laggeadas
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    # Usamos un ajuste linear para estimar el Exponente de Hurst
    poly = polyfit(log(lags), log(tau), 1)
    # Obtenemos el Hurst exponent del output de polyfit
    return poly[0]*2.0

### Creamos un Gometric Brownian Motion, una serie con reversion a la madia y una con tendencia

In [10]:
gbm = log(cumsum(randn(100000))+1000)
mr = log(randn(100000)+1000)
tr = log(cumsum(randn(100000)+1)+1000)

### Imprimimos el Output del Hurst Exponent para cada serie de arriba, asi tambien la del MERVAL para ver a que tipo de serie se asemeja

In [11]:
print("Hurst(GBM): %s" % hurst(gbm))
print("Hurst(MR): %s" % hurst(mr))
print("Hurst(TR): %s" % hurst(tr))
print("Hurst(MERV): %s" % hurst(MERV))

Hurst(GBM): 0.5050424010679435
Hurst(MR): -0.0003196902786829352
Hurst(TR): 0.9585081560083372
Hurst(MERV): 0.45707976988379456


**Conclusion: Observamos que el exponente de Hurst del MERVAL es relativamente cercado a los valores de 0.5. Lo cual explica que el comportamiento del indice es mas similiar al de un GBM, random walk, que a una serie con reversion a la media o con tendencia.**